In [ ]:
# import sys
# sys.path.insert(0, 'driftpy/src/')
# import driftpy
# driftpy.__file__

In [ ]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
pd.options.plotting.backend = "plotly"

# LOAD devnet environment

In [ ]:
import os
from driftpy.clearing_house import ClearingHouse
from driftpy.clearing_house_user import ClearingHouseUser

from arb_helpers import load_position_table, calculate_market_summary

os.environ['ANCHOR_WALLET'] = os.path.expanduser('~/.config/solana/<YOURWALLET>.json')

drift_acct = await ClearingHouse.create_from_env('mainnet')
drift_user = ClearingHouseUser(drift_acct, drift_acct.program.provider.wallet.public_key)

In [ ]:
# pd.DataFrame((await drift_user.get_user_positions_account()).positions)

In [ ]:
# market = await drift_user.clearing_house.get_market(0)
# (await drift_user.get_position_value(0))/1e6

# load user state info

In [ ]:
drift_user_acct = await drift_user.get_user_account()
balance = (drift_user_acct.collateral/1e6)
balance

In [ ]:
position_table_df = await load_position_table(drift_acct)
position_table_df

In [ ]:
QUOTE_PRECISION = 1e6
async def get_margin_account_info(drift_user):
    drift_user_acct = await drift_user.get_user_account()
    margin_info = {}
    margin_info['total_collateral'] = (await drift_user.get_total_collateral())/QUOTE_PRECISION
    margin_info['collateral'] = (drift_user_acct.collateral/QUOTE_PRECISION)
    margin_info['unrealised_pnl'] = (await drift_user.get_unrealised_pnl(0))/QUOTE_PRECISION
    # margin_info['margin_ratio'] = await drift_user.get_margin_ratio()
    margin_info['leverage'] = await drift_user.get_leverage()
    # margin_info['free_collateral'] = (await drift_user.get_free_collateral())/QUOTE_PRECISION
    return pd.Series(margin_info)

await get_margin_account_info(drift_user)

In [ ]:
drift_state = await drift_acct.get_state_account()

# example deposit/withdrawl
USDC_AMOUNT = 1
await drift_acct.deposit_collateral(int(USDC_AMOUNT * QUOTE_PRECISION), drift_state.collateral_vault)
await drift_acct.withdraw_collateral(int(USDC_AMOUNT * QUOTE_PRECISION), drift_state.collateral_vault)

# load predicted funding

In [ ]:
markets = await drift_acct.get_markets_account()
market_summary = await calculate_market_summary(markets)

In [ ]:
market_summary

In [ ]:
position_table_df

In [ ]:
await drift_acct.close_position(0)

In [ ]:
from driftpy.types import PositionDirection
# open a $1 LONG position in the most attractive markets for next funding
market_to_trade = market_summary.sort_values('next_funding_rate(%)').iloc[0].market_index

await drift_acct.open_position(PositionDirection.LONG(), int(1 * 1e6), market_to_trade)

In [ ]:
from driftpy.constants.config import CONFIG
CONFIG['mainnet']